In [1]:
import json

import pandas as pd
import featuretools as ft


with open('cvas_data.json') as f:
    d = json.load(f)
    records = [loan for dd in d['data'] for loan in dd['loans']]
    df = pd.DataFrame.from_records(records, )
    
df

,customer_ID,loan_date,amount,fee,loan_status,term,annual_income
0,1090,15/11/2021,2426,199,0,long,41333
1,3565,07/03/2021,2153,53,0,short,76498
2,3565,06/08/2021,1538,89,1,long,76498
3,3565,30/09/2021,2997,24,0,short,76498
4,3565,09/03/2021,2184,38,0,long,76498
...,...,...,...,...,...,...,...
146,4276,09/04/2019,884,125,0,short,50915
147,447,18/12/2019,438,168,1,long,62868
148,4583,06/06/2020,1672,73,1,short,64470
149,4665,26/09/2019,2555,26,1,long,33923


In [2]:
datetime_cols = ['loan_date']
numeric_cols = ['amount', 'fee', 'annual_income']
categorical_cols = ['loan_status', 'term']

df[datetime_cols] = df[datetime_cols].apply(pd.to_datetime, format='%d/%m/%Y')
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)
df[categorical_cols] = df[categorical_cols].apply(pd.Categorical)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_ID    151 non-null    object        
 1   loan_date      151 non-null    datetime64[ns]
 2   amount         151 non-null    int64         
 3   fee            151 non-null    int64         
 4   loan_status    151 non-null    category      
 5   term           151 non-null    category      
 6   annual_income  151 non-null    int64         
dtypes: category(2), datetime64[ns](1), int64(3), object(1)
memory usage: 6.6+ KB


In [3]:
customers = df[['customer_ID', 'annual_income']].drop_duplicates(subset=['customer_ID'])
loans = df.drop(columns=['annual_income']).reset_index(names='loan_ID')

In [4]:
es = ft.EntitySet(
    id='customer_loans',
    dataframes={
        'customers': (customers, 'customer_ID'),
        'loans': (loans, 'loan_ID', 'loan_date')
    },
    relationships=[
        ('customers', 'customer_ID', 'loans', 'customer_ID')
    ]
)

In [5]:
es['loans'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
loan_ID,Integer,['index']
customer_ID,Unknown,['foreign_key']
loan_date,Datetime,['time_index']
amount,Integer,['numeric']
fee,Integer,['numeric']
loan_status,Categorical,['category']
term,Categorical,['category']


In [6]:
es['customers'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
customer_ID,Unknown,['index']
annual_income,Integer,['numeric']


In [7]:
primitives_reference = ft.list_primitives()
primitives_reference.head()

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,count_outside_range,aggregation,False,False,Determines the number of values that fall outs...,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = IntegerNullable)...
1,percent_true,aggregation,True,False,Determines the percent of `True` values.,<ColumnSchema (Logical Type = BooleanNullable)...,<ColumnSchema (Logical Type = Double) (Semanti...
2,mean,aggregation,True,True,Computes the average for a list of values.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
3,count,aggregation,True,True,"Determines the total number of values, excludi...",<ColumnSchema (Semantic Tags = ['index'])>,<ColumnSchema (Logical Type = IntegerNullable)...
4,mode,aggregation,False,False,Determines the most commonly repeated value.,<ColumnSchema (Semantic Tags = ['category'])>,None


In [8]:
# Features for each customer

customer_features, customer_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='customers',
    agg_primitives=[
        # 'sum',
        # 'std',  # produces nulls
        'max',
        # 'skew',  # produces nulls
        'min',
        'mean',
        'count',
        'percent_true',
        'num_unique',
        'mode',
    ],
    trans_primitives=[
        'year',
        'month',
        'day',
        'day_of_year',
        'distance_to_holiday',
        'is_month_end',
        'is_month_start',
        'time_since_previous',
    ],
    max_depth=2,
)


customer_features

,annual_income,COUNT(loans),MAX(loans.amount),MAX(loans.fee),MEAN(loans.amount),MEAN(loans.fee),MIN(loans.amount),MIN(loans.fee),MODE(loans.loan_status),MODE(loans.term),...,MODE(loans.DAY(loan_date)),MODE(loans.DAY_OF_YEAR(loan_date)),MODE(loans.MONTH(loan_date)),MODE(loans.YEAR(loan_date)),NUM_UNIQUE(loans.DAY(loan_date)),NUM_UNIQUE(loans.DAY_OF_YEAR(loan_date)),NUM_UNIQUE(loans.MONTH(loan_date)),NUM_UNIQUE(loans.YEAR(loan_date)),PERCENT_TRUE(loans.IS_MONTH_END(loan_date)),PERCENT_TRUE(loans.IS_MONTH_START(loan_date))
customer_ID,,,,,,,,,,,,,,,,,,,,,
1090,41333,1,2426.0,199.0,2426.000000,199.000000,2426.0,199.0,0,long,...,15,319,11,2021,1,1,1,1,0.000000,0.0
3565,76498,7,2997.0,146.0,2000.714286,70.857143,207.0,24.0,0,short,...,7,66,3,2021,6,7,5,3,0.142857,0.0
1159,46330,1,2581.0,117.0,2581.000000,117.000000,2581.0,117.0,1,short,...,20,354,12,2021,1,1,1,1,0.000000,0.0
1436,58256,2,2700.0,186.0,1922.500000,184.000000,1145.0,182.0,0,long,...,16,47,2,2019,2,2,2,2,0.000000,0.0
1512,67215,2,2114.0,78.0,1162.000000,76.500000,210.0,75.0,0,long,...,24,88,3,2019,2,2,2,2,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4276,50915,6,2243.0,161.0,1730.500000,109.333333,884.0,39.0,0,long,...,5,77,6,2020,6,6,4,3,0.166667,0.0
447,62868,1,438.0,168.0,438.000000,168.000000,438.0,168.0,1,long,...,18,352,12,2019,1,1,1,1,0.000000,0.0
4583,64470,1,1672.0,73.0,1672.000000,73.000000,1672.0,73.0,1,short,...,6,158,6,2020,1,1,1,1,0.000000,0.0


In [9]:
customer_defs

[<Feature: annual_income>,
 <Feature: COUNT(loans)>,
 <Feature: MAX(loans.amount)>,
 <Feature: MAX(loans.fee)>,
 <Feature: MEAN(loans.amount)>,
 <Feature: MEAN(loans.fee)>,
 <Feature: MIN(loans.amount)>,
 <Feature: MIN(loans.fee)>,
 <Feature: MODE(loans.loan_status)>,
 <Feature: MODE(loans.term)>,
 <Feature: NUM_UNIQUE(loans.loan_status)>,
 <Feature: NUM_UNIQUE(loans.term)>,
 <Feature: MAX(loans.DISTANCE_TO_HOLIDAY(loan_date))>,
 <Feature: MAX(loans.TIME_SINCE_PREVIOUS(loan_date))>,
 <Feature: MEAN(loans.DISTANCE_TO_HOLIDAY(loan_date))>,
 <Feature: MEAN(loans.TIME_SINCE_PREVIOUS(loan_date))>,
 <Feature: MIN(loans.DISTANCE_TO_HOLIDAY(loan_date))>,
 <Feature: MIN(loans.TIME_SINCE_PREVIOUS(loan_date))>,
 <Feature: MODE(loans.DAY(loan_date))>,
 <Feature: MODE(loans.DAY_OF_YEAR(loan_date))>,
 <Feature: MODE(loans.MONTH(loan_date))>,
 <Feature: MODE(loans.YEAR(loan_date))>,
 <Feature: NUM_UNIQUE(loans.DAY(loan_date))>,
 <Feature: NUM_UNIQUE(loans.DAY_OF_YEAR(loan_date))>,
 <Feature: NUM_UNIQ